In [1]:
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import MetadataCatalog, DatasetCatalog
import detectron2.data.transforms as T
from detectron2.structures.image_list import ImageList
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.modeling.box_regression import Box2BoxTransform
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetMapper
from detectron2.data import (
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.data.samplers import TrainingSampler
from detectron2.modeling.proposal_generator.proposal_utils import add_ground_truth_to_proposals
from detectron2.modeling.sampling import subsample_labels
from detectron2.structures import Boxes, Instances, pairwise_iou
from detectron2.modeling.matcher import Matcher
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.utils.visualizer import Visualizer

from fsdet.modeling.roi_heads.fast_rcnn import FastRCNNOutputs
from fsdet.evaluation import (
    COCOEvaluator, DatasetEvaluators, LVISEvaluator, PascalVOCDetectionEvaluator, verify_results)
from fsdet.evaluation import (
    DatasetEvaluator,
    inference_on_dataset,
    print_csv_format,
    verify_results,
)
import cv2
import torch, torchvision
import logging
import os
from sklearn import svm
from joblib import dump, load
from detectron2.layers import nonzero_tuple
import numpy as np

logging.basicConfig(level=logging.INFO)

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



# Load pretrained detection model and Prediect directly

## Load models of DNN and SVM

In [2]:
cfg = get_cfg()
cfg.merge_from_file('configs/test_unfreeze_lastfews.yaml')
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 100
cfg.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.8
model = build_model(cfg)  # returns a torch.nn.Module
print(cfg.INPUT.FORMAT)

model.eval()
metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])

# ckpt_file = 'checkpoints/coco/base_model/model_final.pth'
# ckpt_file = 'checkpoints/coco/faster_rcnn/30shot_person_freeze_last_cos/model_final.pth'
# ckpt_file = 'checkpoints/coco/faster_rcnn/30shot_person_freeze_last_fc/model_final.pth'
# ckpt_file = 'checkpoints/coco/faster_rcnn/30shot_person_unfreeze_lastfews/model_final.pth'
# ckpt_file = 'checkpoints/coco/faster_rcnn/30shot_person_unfreeze_whole/model_0015999.pth'
ckpt_file = 'checkpoints/coco/faster_rcnn/30shot_airplane_unfreeze_whole/model_0029999.pth'
DetectionCheckpointer(model).load(ckpt_file)

# clf = load('svm_results/svm_model_finetuned_prop_base.joblib') 
# clf = load('svm_results/svm_model_finetuned_prop_lastfew.joblib') 
# clf = load('svm_results/svm_model_finetuned_prop_whole.joblib') 
shots_num = 500
class_name = 'airplane'
class_id = 5
# [1, 2, 3, 5, 17, 18, 19, 20, 62]: # [person, bicycle, car, airplane, cat, dog, horse, sheep, chair]

INFO:fvcore.common.checkpoint:Loading checkpoint from checkpoints/coco/faster_rcnn/30shot_airplane_unfreeze_whole/model_0029999.pth


BGR


## Register dataset

In [3]:
from detectron2.data.datasets import register_coco_instances
json_dir = 'datasets/coco_experiments/seed1/full_box_{}shot_{}_train.json'.format(shots_num, class_name)
image_dir = 'datasets/coco/trainval2014'
trainset_name = "{}shot_{}_train".format(shots_num, class_name)
register_coco_instances(trainset_name, {}, json_dir, image_dir)


## Difference between some dataloader

In [4]:
# data_loader_train = build_detection_train_loader(cfg)
# data_loader_train_it = iter(data_loader_train)
# data = next(data_loader_train_it)
# # batch size = 16 as is in the config file
# print(len(data))
# print(data[0]['image'].shape)
# print(data)

In [5]:
# data_loader_test = build_detection_test_loader(cfg, "1000shot_person_test")
# data_loader_test_it = iter(data_loader_test)


In [6]:
# data = next(data_loader_test_it)
# print(len(data))
# print(data[0]['image'].shape)
# print(data)

In [7]:
transform_gen = T.ResizeShortestEdge(
            [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
            cfg.INPUT.MAX_SIZE_TEST,
            )
batch_size = 10
data_loader = build_detection_train_loader(DatasetCatalog.get(trainset_name),
                                           mapper=DatasetMapper(cfg, is_train=True, augmentations=[transform_gen]),
                                           total_batch_size = batch_size)
data_loader_it = iter(data_loader)
data = next(data_loader_it)
print(len(data))
print(data[0]['image'].shape)
# print(data)

# the type of dataloader is different 'AspectRatio Grouped Dataset' (don't know the reason) 

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

INFO:detectron2.data.datasets.coco:Loaded 274 images in COCO format from datasets/coco_experiments/seed1/full_box_500shot_airplane_train.json
INFO:detectron2.data.dataset_mapper:[DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=[800, 800], max_size=1333)]
INFO:detectron2.data.common:Serializing 274 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 0.40 MiB


10
torch.Size([3, 800, 1196])


# Run detector step by step and get X and y for SVM training

In [8]:
assert len(cfg.MODEL.PIXEL_MEAN) == len(cfg.MODEL.PIXEL_STD)
num_channels = len(cfg.MODEL.PIXEL_MEAN)
device = 'cuda'
pixel_mean = (
    torch.Tensor(cfg.MODEL.PIXEL_MEAN)
    .to(device)
    .view(num_channels, 1, 1)
)
pixel_std = (
    torch.Tensor(cfg.MODEL.PIXEL_STD)
    .to(device)
    .view(num_channels, 1, 1)
)
normalizer = lambda x: (x - pixel_mean) / pixel_std

In [9]:

from detectron2.utils.events import EventStorage
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

box2box_transform = Box2BoxTransform(
            weights=cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS
        )
smooth_l1_beta = cfg.MODEL.ROI_BOX_HEAD.SMOOTH_L1_BETA
test_score_thresh        = cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST
# test_score_thresh        = 0.5
test_nms_thresh          = cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST
test_detections_per_img  = cfg.TEST.DETECTIONS_PER_IMAGE

iter_num = shots_num//batch_size

feature_vec_list = []
proposals_with_gt = []
with EventStorage() as storage:
    with torch.no_grad():
#         for idx, inputs in enumerate(data_loader_train):
#             batched_inputs = inputs
        for idx in range(iter_num):
            batched_inputs = next(data_loader_it)

            # Normalize, pad and batch the input images. (Preprocess_image)
            images = [x["image"].to('cuda') for x in batched_inputs]
            images = [normalizer(x) for x in images]
            images = ImageList.from_tensors(
                images, model.backbone.size_divisibility
            )

            # forward
            features = model.backbone(images.tensor)
    #         print('features shape:', features['p3'].shape)
            proposals, _ = model.proposal_generator(images, features)
    #         print('proposal num per img:', proposals_list[0].objectness_logits.shape)

            # run roi_heads step by step
            targets = [d['instances'].to('cuda') for d in batched_inputs]
            proposals = model.roi_heads.label_and_sample_proposals(proposals, targets)

            box_features = model.roi_heads.box_pooler(
                [features[f] for f in ["p2", "p3", "p4", "p5"]], [x.proposal_boxes for x in proposals]
            )
    #         print(box_features.shape)
            box_features = model.roi_heads.box_head(box_features)
#             print(box_features.shape) # torch.Size([5120, 1024])

            # For SVM training: X and y
            feature_vec_list.extend(box_features)
            proposals_with_gt.extend(proposals)


In [10]:
# print(len(feature_vec_list))
# print(len(proposals_with_gt))
X = torch.vstack(feature_vec_list).cpu().detach().numpy()
y = torch.vstack([p.gt_classes.reshape((-1, 1)) for p in proposals_with_gt]).cpu().detach().numpy().ravel()
y[y==class_id-1] =  0
pos_num = y[y==0].shape[0]
neg_num = y[y==1].shape[0]
print('positive : negative = ', pos_num, ' : ', neg_num)
print('X shape = ', X.shape)
print('y shape = ', y.shape)

positive : negative =  723  :  2277
X shape =  (3000, 1024)
y shape =  (3000,)


# Fit SVM Model

In [12]:
file_name = 'svm_{}_{}_{}.joblib'.format(ckpt_file.split('/')[-2], shots_num, class_name)
print(file_name)

svm_30shot_airplane_unfreeze_whole_500_airplane.joblib


In [13]:
clf = svm.SVC(probability=True)
clf.fit(X, y)
ckpt_file.split('/')[-2]
dump(clf, 'svm_results_0216/' + file_name) 

['svm_results_0216/svm_30shot_airplane_unfreeze_whole_500_airplane.joblib']